In [23]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 776.7 MB 1.7 kB/s eta 0:00:01   |▏                               | 4.5 MB 1.5 MB/s eta 0:08:43     |▋                               | 13.9 MB 853 kB/s eta 0:14:55     |▋                               | 15.7 MB 1.2 MB/s eta 0:10:59     |▊                               | 16.3 MB 1.4 MB/s eta 0:08:45     |▊                               | 18.0 MB 6.7 MB/s eta 0:01:54     |▉                               | 19.9 MB 1.4 MB/s eta 0:09:02     |█▏                              | 29.0 MB 1.6 MB/s eta 0:07:56     |█▌                              | 37.7 MB 1.7 MB/s eta 0:07:27     |█▊                              | 42.8 MB 1.5 MB/s eta 0:08:20     |██▎                             | 54.0 MB 1.5 MB/s eta 0:08:01     |██▍                             | 59.1 MB 1.6 MB/s eta 0:07:35     |██▋                             | 64.6 MB 1.6 MB/s eta 0:07:32     |███                             | 

In [3]:
import pandas as pd 
train=pd.read_csv('train.csv',index_col=0)
test=pd.read_csv('test_x.csv',index_col=0)
def tp_score(train):
    train['diligence']=round((train['tp03']+ 8-train['tp08'])/2)
    train['Affinity']=round((train['tp07']+ 8-train['tp02'])/2)
    train['emotional stability']=round((train['tp09']+ 8-train['tp04'])/2)
    train['openness']=round((train['tp05']+ 8-train['tp10'])/2)
    train['extroversion']=round((train['tp01']+ 8-train['tp06'])/2)
    train.drop([ 'tp01', 'tp02', 'tp03', 'tp04', 'tp05','tp06', 'tp07', 'tp08', 'tp09', 'tp10'],axis=1,inplace=True)
    return train
train=tp_score(train)
test=tp_score(test)

train.age_group=train.age_group.apply(lambda x : int(x.replace('0s','')) )
test.age_group=test.age_group.apply(lambda x : int(x.replace('0s','')) )

train=train.drop(['QaE','QbE','QcE','QdE','QeE','QfE','QgE','QhE','QiE','QjE','QkE','QlE','QmE','QnE','QoE','QpE','QqE','QrE','QsE','QtE'], axis=1)
test=test.drop(['QaE','QbE','QcE','QdE','QeE','QfE','QgE','QhE','QiE','QjE','QkE','QlE','QmE','QnE','QoE','QpE','QqE','QrE','QsE','QtE'], axis=1)

train['race']=train.race.apply(lambda x : 1 if x=='White' else 0 ) 
train['hand']=train.hand.apply(lambda x : 1 if x==1 else 0 )
test['race']=test.race.apply(lambda x : 1 if x=='White' else 0 )
test['hand']=test.hand.apply(lambda x : 1 if x==1 else 0 )

train1=pd.concat([train[train.columns.difference(['gender', 'religion','voted'])] ,pd.get_dummies(train[['gender','religion']])],axis=1)
train1.drop(['gender_Male'],axis=1,inplace=True)

In [2]:
# score 용 
# test=pd.concat([test[test.columns.difference(['education','gender', 'religion','married','urban'])] ,pd.get_dummies(test[['education','gender',  'religion','married','urban']])],axis=1)
# test.drop(['gender_Male'],axis=1,inplace=True)
# test=scaler.fit_transform(test)
# test=pd.DataFrame(test)

In [4]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
train_data=[]
for i in range(len(train1)):
    train_data.append(train1.iloc[train1.index==i,:].values[0].tolist())    
train_data = np.array(train_data)  

encoder = OneHotEncoder()
one_hot_encoded_train = encoder.fit_transform(train_data.reshape(-1,1))
one_hot_encoded_train = one_hot_encoded_train.toarray()
x_train=one_hot_encoded_train[:35000]
y_train=np.array(train['voted'])[:35000]

x_test=one_hot_encoded_train[35000:]
y_test=np.array(train['voted'])[35000:]

In [24]:
import numpy as np
import torch
train_data=[]
for i in range(len(train1)):
    train_data.append(train1.iloc[train1.index==i,:].values[0].tolist())    
train_data = torch.tensor(train_data)
test =torch.tensor(train['voted'])

In [49]:
np.asarray(test).astype('str').shape

()

In [26]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1. 
    return results

x_train = vectorize_sequences(train_data) 
y_train = np.array(train['voted'])

# x_test = vectorize_sequences(train_data)
# y_test =np.array(train['voted'])

# y_train = np.asarray(train_labels).astype('float32')
# y_test = np.asarray(test_labels).astype('float32')

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [50]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(16, input_dim=27, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='sgd', loss='mse', metrics=[tensorflow.keras.metrics.AUC()])
model.fit(np.asarray(train_data).astype('float32').reshape(45532, 62,1), np.asarray(test).astype('str'), epochs=4, batch_size=4)
# results = model.evaluate(x_test, y_test)
# y_predicted = model.predict(x_test)

IndexError: list index out of range

In [ ]:
df = pd.DataFrame(data={'voted':lgb.predict(test)})
df.reset_index(inplace=True)
# df.iloc[:, 1:] = prediction
# df.to_csv('./Result/{}.csv'.format(datetime.now().strftime('%m%d-%H%M')), index=False)